주요 어휘 추출 전처리 후 페이지 추천 기능 적용

1. 전처리 코드 완성(디버깅 포함, 워드클라우드 생성)

※ 라이브러리 및 프레임워크 필요시 설치

!pip install konlpy

!pip install JPype1

!pip install wordcloud matplotlib

!pip install --upgrade pip

!pip install --upgrade Pillow

In [1]:
import pandas as pd
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [2]:
# 데이터 로드
df = pd.read_csv("data/wanted_crawling_all_data.csv", index_col=None, 
                 parse_dates=['Title', 'URL'], encoding='cp949')

In [3]:
# 불용어 리스트를 파일에서 읽어오기
with open('stopwords-ko.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

In [4]:
# 형태소 분석기 초기화
okt = Okt()

In [5]:
# 전처리 함수 정의 (명사 추출 및 불용어 제거)
def preprocess_text(text):
    if isinstance(text, str):  # 문자열 데이터만 처리
        text = text.lower()
        text = re.sub(r'\d+', '', text)  # 숫자 제거
        text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
        # 명사 추출
        nouns = okt.nouns(text)
        # 불용어 제거
        filtered_nouns = [noun for noun in nouns if noun not in stopwords and len(noun) > 1]
        return ' '.join(filtered_nouns)
    return ''

In [6]:
# TF-IDF 기반 중요 단어 추출 함수
def get_important_words(column_text, n=50):
    vectorizer = TfidfVectorizer(max_features=n, max_df=0.85, min_df=1) 
    X = vectorizer.fit_transform(column_text)
    if X.shape[0] == 0:  # 문서가 없는 경우
        return []
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = X.toarray().sum(axis=0)
    word_score_pairs = sorted(zip(feature_names, tfidf_scores), key=lambda x: x[1], reverse=True)
    return word_score_pairs

In [7]:
# 각 열에 대해 전처리 및 TF-IDF 기반 중요 단어 추출 (URL 칼럼 제외)
important_words_dict = {}

In [8]:
for column in df.columns:
    # 해당 칼럼은 처리하지 않음
    if column in ['Title', 'Company', 'Career', 'Deadline', 'Location', 'Duty', 'URL']:
        continue
    if df[column].dtype == 'object':  # 문자열 데이터에 대해서만 처리
        # 전처리 및 명사 추출
        df[column] = df[column].apply(preprocess_text)
        column_text = df[column].dropna().tolist()  # NaN 값 제거 및 리스트로 변환
        
        # 텍스트 샘플 출력 (디버깅용)
        print(f"Column: {column}")
        print("Sample Texts:")
        print(column_text[:5])  # 상위 5개 텍스트 샘플 출력
        
        if len(column_text) > 0:  # 데이터가 있는 경우에만 처리
            # TF-IDF 기반 중요 단어 추출
            important_words = get_important_words(column_text, n=50)
            if important_words:  # 중요 단어가 추출된 경우에만 처리
                important_words_dict[column] = important_words

Column: Work
Sample Texts:
['주요 책임 글로벌 유저 대상 게임 플랫폼 개발 플랫폼 서비스 운영 지속 개선 게임 서비스 백오피스 기능 개발 도화주 환경 프로젝트 일정 집중 기간 존재', '합류 토스 레이스 오프라인 결제 시장 디지털 혁신 만들기 위해 매장 운영 문제 해결 레이스 초창 멤버 로서 오프라인 결제 시장 마주 도전 문제 처음 고민 개발 사장 대시보드 홈페이지 웹사이트 토스 결제 단말기 서비스 웹뷰 모든 플랫폼 기술 이용 사용자 향상 위해 토스 공통 라이브러리 제작 기여', '프레임워크 개발 유지 보수 자사 솔루션 백오피스 비롯 마이크로 사이트 개발 유지 보수 활용 데이터 바인 렌더링 설계 구현', '모바일 게임 마피아 서버 개발자 채팅 모바일 온라인 게임 서버 개발', '제휴 파트너 효과 테크 서비스 개발 성장 중인 트래블 월렛 차세대 외환 결제 백오피스 개발 퍼블릭 웹서비스 개발 운영 백오피스 사내 운영 개발 운영']
Column: Qualification
Sample Texts:
['필수 자격 요건 학력 전공 무관 개발 보유 유저 서비스 개발 보유 이해 지식 보유 라이브러리 프레임워크 활용 개발 관심 선호 자격 요건 개발 언어 로서 이해 언어 강화 위해 노력 환경 모바일 웹뷰 보유 기술 이해 렌더링 개발 배포 스크립트 작성 구축 운용 지식 협업 실무 보유 디자인 시스템 실무 보유 웹사이트 성능 최적화 분필 효과 커뮤니케이션 여러 유관 부서 외부 관계자 협업 협력 사고 통합 관점 목표 의식 주도 추진 문제 파악 분석 적극 선제 문제해결 책임감 결과물 품질 완성 기준', '분과 프레임워크 사용 개발 주도 문제 발견 분석 해결 이용 정적 타입 분석', '이해 프론트엔드 프레임워크 이해 개발 의사소통 협업', '자료구조 알고리즘 운영체제 네트워크 컴퓨터 학적 지식 언어 지식 설계 개발 포트폴리오 제출 필수', '관련 개발 실력 보유 활용 어플리케이션 평소 사용 사용 커뮤니케이션']
Column: Addition
Sample Texts

In [ ]:
# 각 칼럼별 중요 단어를 CSV 파일로 저장
for column, words in important_words_dict.items():
    df_words = pd.DataFrame(words, columns=['Word', 'Score'])
    df_words.to_csv(f'{column}_important_words.csv', index=False, encoding='utf-8-sig')

In [9]:
# 전처리된 텍스트 확인 (디버깅용)
for column in df.columns:
    if column in important_words_dict:
        print(f"Column: {column}")
        print(df[column].head())

Column: Work
0    주요 책임 글로벌 유저 대상 게임 플랫폼 개발 플랫폼 서비스 운영 지속 개선 게임 ...
1    합류 토스 레이스 오프라인 결제 시장 디지털 혁신 만들기 위해 매장 운영 문제 해결...
2    프레임워크 개발 유지 보수 자사 솔루션 백오피스 비롯 마이크로 사이트 개발 유지 보...
3                모바일 게임 마피아 서버 개발자 채팅 모바일 온라인 게임 서버 개발
4    제휴 파트너 효과 테크 서비스 개발 성장 중인 트래블 월렛 차세대 외환 결제 백오피...
Name: Work, dtype: object
Column: Qualification
0    필수 자격 요건 학력 전공 무관 개발 보유 유저 서비스 개발 보유 이해 지식 보유 ...
1            분과 프레임워크 사용 개발 주도 문제 발견 분석 해결 이용 정적 타입 분석
2                         이해 프론트엔드 프레임워크 이해 개발 의사소통 협업
3    자료구조 알고리즘 운영체제 네트워크 컴퓨터 학적 지식 언어 지식 설계 개발 포트폴리...
4                관련 개발 실력 보유 활용 어플리케이션 평소 사용 사용 커뮤니케이션
Name: Qualification, dtype: object
Column: Addition
0    조직 문화 자율 출퇴근 무제한 연차 휴가 스스로 관리 의견 교환 상호 신뢰 근거 수...
1    이력서 작성 추천 플랫폼 관련 공통 서비스 공통 라이브러리 구현 운영 해당 진행 이...
2        이해 관심 사용 성능 최적화 관심 이해 테스트 자동화 단위 테스트 라이브러리 모듈
3    게임 웹앱 온라인 서비스 백엔드 개발 용량 부하 시스템 설계 운영 리눅스 운영 서버...
4    배포 클라우드 배포 환경 이해 프론트엔드 개발 방법 이해 개발 디자인 시스템 재사용...
Name: Addition, dtype: object
Column: Welfare
0               

In [ ]:
# 전처리된 데이터프레임 저장
df.to_csv('preprocessed_data_all.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 워드클라우드 생성 및 저장 함수
def create_wordcloud(words, column_name):
    word_freq = dict(words)  # 중요 단어와 점수를 딕셔너리 형태로 변환
    wordcloud = WordCloud(font_path='C:/Windows/Fonts/malgun.ttf',  # 한글 폰트 경로
                          width=800, height=400, 
                          background_color='white').generate_from_frequencies(word_freq)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Word Cloud for {column_name}')
    plt.savefig(f'{column_name}_wordcloud.png', format='png')
    plt.show()

# 각 칼럼별 워드클라우드 생성 및 저장
for column, words in important_words_dict.items():
    create_wordcloud(words, column)

2. 전처리 데이터 데이터베이스 적재

In [10]:
df.head()

,Title,Company,Career,Work,Qualification,Addition,Welfare,Skill,Tag,Deadline,Location,Duty,URL
0,플랫폼 웹프론트엔드 개발,하이브아이엠,경력 3-7년,주요 책임 글로벌 유저 대상 게임 플랫폼 개발 플랫폼 서비스 운영 지속 개선 게임 ...,필수 자격 요건 학력 전공 무관 개발 보유 유저 서비스 개발 보유 이해 지식 보유 ...,조직 문화 자율 출퇴근 무제한 연차 휴가 스스로 관리 의견 교환 상호 신뢰 근거 수...,,,출산휴가 육아휴직 무제한 연차 상위 인원 설립,상시채용,서울특별시 강남구 테헤란로108길 42,개발,https://www.wanted.co.kr/wd/234936
1,Frontend Developer,토스플레이스,경력 1-15년,합류 토스 레이스 오프라인 결제 시장 디지털 혁신 만들기 위해 매장 운영 문제 해결...,분과 프레임워크 사용 개발 주도 문제 발견 분석 해결 이용 정적 타입 분석,이력서 작성 추천 플랫폼 관련 공통 서비스 공통 라이브러리 구현 운영 해당 진행 이...,일해 자율 책임 문화 토스 구성 조직 정보 공유 수평 문화 구성원 위임 신뢰 바탕 ...,,통신비 보너스 커피 스낵바 재택근무 편의 시설 단체 보험 상위 인원 설립,상시채용,"서울특별시 강남구 테헤란로 131, 17층",개발,https://www.wanted.co.kr/wd/207573
2,Frontend 개발,인더코어비즈니스플랫폼,신입,프레임워크 개발 유지 보수 자사 솔루션 백오피스 비롯 마이크로 사이트 개발 유지 보...,이해 프론트엔드 프레임워크 이해 개발 의사소통 협업,이해 관심 사용 성능 최적화 관심 이해 테스트 자동화 단위 테스트 라이브러리 모듈,근무 형태 개월 계약 기간 평가 결과 정규직 전환 계약 기간 서로 서로 준비 생각 ...,,설립 상위 유연 근무 커피 스낵바 식대 편의 시설 자기계발 장비,상시채용,"영등포구 여의대로 108, 파크원타워2, 46층",개발,https://www.wanted.co.kr/wd/235725
3,[마피아42] 서버개발자,팀42,신입,모바일 게임 마피아 서버 개발자 채팅 모바일 온라인 게임 서버 개발,자료구조 알고리즘 운영체제 네트워크 컴퓨터 학적 지식 언어 지식 설계 개발 포트폴리...,게임 웹앱 온라인 서비스 백엔드 개발 용량 부하 시스템 설계 운영 리눅스 운영 서버...,자율 출퇴근 점심 연차 운동 최고급 장비 밀러 의자 아이맥 무제한 간식 교통 공덕역...,,장비 자기계발 커피 스낵바 상위 설립 아시아 태평양 성장 인원,상시채용,마포구 새창로7 SNU장학빌딩 12층,개발,https://www.wanted.co.kr/wd/234496
4,프론트엔드 개발자 (Junior),트래블월렛,경력 2-3년,제휴 파트너 효과 테크 서비스 개발 성장 중인 트래블 월렛 차세대 외환 결제 백오피...,관련 개발 실력 보유 활용 어플리케이션 평소 사용 사용 커뮤니케이션,배포 클라우드 배포 환경 이해 프론트엔드 개발 방법 이해 개발 디자인 시스템 재사용...,개발 환경 개발 환경 배포 환경 환경 시차 출퇴근 제로 사이 출근 연차 차시 눈치 ...,,사내 동호회 휴가 건강검진 커피 스낵바 단체 보험 자기계발 유연 근무 설립 누적 투...,상시채용,"서울시 강남구 테헤란로 418, 12층(스파크플러스 선릉2호점)",개발,https://www.wanted.co.kr/wd/232778


!pip install pymysql

In [12]:
import pymysql

# 기존 테이블 삭제
def db_delete():
    # MariaDB 연결
    conn = pymysql.connect(host='127.0.0.1', user='root', password='root', db='dbmaria', charset='utf8')
    
    try:
        # 커서 생성
        cur = conn.cursor()
        
        # 테이블 삭제
        cur.execute('DROP TABLE IF EXISTS job_postings')
        
        # 변경사항 커밋
        conn.commit()
    finally:
        # 연결 종료
        conn.close()

# 함수 호출
db_delete()

In [15]:
## 1. 데이터 베이스에 테이블 생성

# MariaDB 연결
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='root', db='dbmaria', charset='utf8')

# 커서 생성
cur = conn.cursor()

# 테이블 생성
cur.execute('''
    CREATE TABLE IF NOT EXISTS job_postings (
        Title VARCHAR(255),
        Company VARCHAR(255),
        Career VARCHAR(255),
        Work TEXT,
        Qualification TEXT,
        Addition TEXT,
        Welfare TEXT,
        Skill TEXT,
        Tag TEXT,
        Deadline VARCHAR(255),
        Location VARCHAR(255),
        Duty VARCHAR(255),
        URL VARCHAR(255),
        PRIMARY KEY (URL)
    )
''')

# 변경사항 커밋
conn.commit()

# 연결 종료
conn.close()

In [16]:
## 2. 데이터 프레임을 데이터베이스에 삽입

# MariaDB 연결
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='root', db='dbmaria', charset='utf8')

# 커서 생성
cur = conn.cursor()

# 데이터 삽입
for index, row in df.iterrows():
    cur.execute('''
        INSERT INTO job_postings (
            Title, Company, Career, Work, Qualification, Addition, Welfare, Skill, Tag, Deadline, Location, Duty, URL
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', (
        row['Title'], row['Company'], row['Career'], row['Work'], row['Qualification'],
        row['Addition'], row['Welfare'], row['Skill'], row['Tag'], row['Deadline'],
        row['Location'], row['Duty'], row['URL']
    ))

# 변경사항 커밋
conn.commit()

# 연결 종료
conn.close()

In [17]:
## 3. 데이터 확인

# MariaDB 연결
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='root', db='dbmaria', charset='utf8')

# 커서 생성
cur = conn.cursor()

# 데이터 조회
cur.execute("SELECT * FROM job_postings")
results = cur.fetchall()

# 결과 출력
for row in results:
    print(row)

# 연결 종료
conn.close()

('[커머스프로덕션] 프로덕트 디자이너(UI/UX Designer)', '미리디', '신입 이상', '문제 정의 문제 해결 분석 설계 디자인 담당 사용자 배포 과정 참여 스쿼드 엔지니어 데이터 프로 디자이너 등로 구성 가지 프로덕트 집중 데이터 분석 고객 활동 지표 데이터 분석 리서치 시장 조사 통해 문제 발견 정의 리서치 결과 보고서 상세 정책 설계 프리 타이핑 디자인 애자 린스타트업 문제 해결', '실행 스스로 목표 설정 주도 바탕 문제 정의 논리 해결 정보 사용자 이해 구조 질문 통해 문제 본질 목적 파악 서비스 기획 지속 발전 정성 사용자 조사 결과 정량 데이터 문제 서비스 개선 서비스 자율성 책임감 가지 항상 노력 제품 분석 디자인 방향성 제안 모든 요소 대하 이유 설계 디자인 고객 이해 라이 글쓰기 지니 개발자 협업 배려 바탕 논리 커뮤니케이션 효과 피드백 디지털 제품 디자인 관련 연구 습관 통해 인사이트 적용 흥미', '정책 이해 정의 서비스 직접 설계 리서치 결과 보고서 상세 정책 설계 디자인 사용 디자인 시스템 이해도 비주 디자인 이해 위계 질서 디자인 디바이스 범용 디자인 패턴 숙지 서비스 적용 대중 디자인 시스템 기본 컴포넌트 이해 활용 디지털 환경 화면 원리 대해 이해 데이터 분석 데이터 분석 리서치 시장 조사 통해 문제 발견 서비스 개선 데이터 이해 사용자 디자인 성장 동료 환경 브랜드 매너 일관 유지 사용자 이해 문장 작성 문화 배경 이해 반영 글로벌 사용자 언어 표현 사용 분과 해결 디자인 집중 본질 문제 해결 기능 추가 집중 동료 피드백 해결 구상 고민 결과물 책임감 지속 성장 발전 욕심 스스로 목표 일정 계획 체계 관리 몰입', '자율 책임 근무 환경 자율 출퇴근 제도 유연 근무시간 재택근무 원격 근무 제도 휴가 사용 경조사 휴가 배움 성장 행복 관련 비용 도서 비용 무제한 직원 행복 행복 개발 외부 교육 기술 세미나 적극 활용 성장 미리 시세 아침 점심 저녁 식사 제휴 카페 포인트 미리 회사 적응 단합 직급 수평 문화 지향 근속

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 4. 데이터 수정

import pymysql

# MariaDB 연결
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password='root', db='dbmaria', charset='utf8')

# 커서 생성
cur = conn.cursor()

# 데이터 수정
cur.execute('''
    UPDATE job_postings
    SET Title = %s, Company = %s
    WHERE URL = %s
''', ('New Title', 'New Company', 'http://example.com'))

# 변경사항 커밋
conn.commit()

# 연결 종료
conn.close()